In [1]:
import numpy as np
import optuna.logging
import pandas as pd

from onnxconverter_common import FloatTensorType
from onnxmltools.convert import convert_sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold, train_test_split

optuna.logging.set_verbosity(optuna.logging.WARNING)

C:\Users\yuki\IdeaProjects\PhishDetector\model\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 特徴量データの読み込み
features_and_label: pd.DataFrame = pd.read_csv("../feature.csv").drop(columns=["domain"])

# featuresとlabelを分割
features = features_and_label.drop("label", axis=1)
labels = features_and_label[["label"]]

In [3]:
# Stratified 5-fold CV

cv_scores_columns = ["accuracy", "TPR", "TNR", "FPR", "FNR", "F1", "AUC", "TP", "TN", "FP", "FN"]
history = np.empty((0, 11))

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, test_idx in kf.split(features, labels):
    x_train: pd.DataFrame = features.iloc[train_idx, :]  # 学習用特徴量
    y_train: pd.DataFrame = labels.iloc[train_idx, 0]  # 学習用ラベル
    x_test: pd.DataFrame = features.iloc[test_idx, :]  # テスト用特徴量
    y_test: pd.DataFrame = labels.iloc[test_idx, 0]  # テスト用ラベル

    # 訓練
    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # 予測
    label_pred_prob = model.predict_proba(x_test)[:, 1]
    label_pred = (label_pred_prob > 0.5).astype(int)

    # 評価指標の計算
    tn, fp, fn, tp = confusion_matrix(y_test, label_pred).flatten()
    acc = accuracy_score(y_test, label_pred)
    tpr = tp / (tp + fn)  # recall
    tnr = tn / (tn + fp)  # specificity
    fpr = fp / (tn + fp)
    fnr = fn / (tp + fn)
    f1 = f1_score(y_test, label_pred)
    auc = roc_auc_score(y_test, label_pred_prob)

    # 評価結果の格納
    history = np.vstack([history, [acc, tpr, tnr, fpr, fnr, f1, auc, tp, tn, fp, fn]])

In [4]:
mean = np.mean(history, axis=0)
history_final = np.vstack([history, mean])

df_cv_scores = pd.DataFrame(history_final, columns=cv_scores_columns, index=["1", "2", "3", "4", "5", "平均"])
df_cv_scores

,accuracy,TPR,TNR,FPR,FNR,F1,AUC,TP,TN,FP,FN
1,0.987437,0.989950,0.984925,0.015075,0.010050,0.987469,0.999015,197.0,196.0,3.0,2.0
2,0.987437,0.994975,0.979899,0.020101,0.005025,0.987531,0.999571,198.0,195.0,4.0,1.0
3,0.984925,0.984925,0.984925,0.015075,0.015075,0.984925,0.999280,196.0,196.0,3.0,3.0
4,0.977387,0.969849,0.984925,0.015075,0.030151,0.977215,0.993889,193.0,196.0,3.0,6.0
5,0.979899,0.994975,0.964824,0.035176,0.005025,0.980198,0.999722,198.0,192.0,7.0,1.0
平均,0.983417,0.986935,0.979899,0.020101,0.013065,0.983468,0.998295,196.4,195.0,4.0,2.6


In [5]:
# 訓練
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, stratify=labels, random_state=42)

# 訓練
model = RandomForestClassifier()
model.fit(x_train, y_train.iloc[:, 0])

importance = pd.DataFrame(
    model.feature_importances_,
    index=features.columns.tolist(),
    columns=['importance']
).sort_values(by='importance', ascending=False)

importance

,importance
scriptTagCount,0.312112
noDomainInInternalLink,0.248225
externalLinkPercentage,0.197236
TagCountInHeadTag,0.081770
googleAnalytics,0.051692
iframeTagCount,0.049138
samePageLinkCount,0.046150
noTitle,0.006899
copy,0.005526
invalidKiyaku,0.000669


In [6]:
# ONNXへの変換
initial_type = [
    ("X", FloatTensorType([None, 11])),
]
onx = convert_sklearn(model=model, initial_types=initial_type)
with open("../onnx_model/rf.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [7]:
import onnx
onnx_model = onnx.load("../onnx_model/rf.onnx")
onnx.checker.check_model(onnx_model)